In [1]:
import numpy as np
import torch
from transformers import T5EncoderModel, T5Tokenizer
import re

# Load model
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False)
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")

# Load into GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model = model.eval()

Some weights of the model checkpoint at Rostlab/prot_t5_xl_uniref50 were not used when initializing T5EncoderModel: ['decoder.block.22.layer.2.DenseReluDense.wo.weight', 'decoder.block.13.layer.2.layer_norm.weight', 'decoder.block.23.layer.0.SelfAttention.q.weight', 'decoder.block.15.layer.1.EncDecAttention.q.weight', 'decoder.block.16.layer.0.SelfAttention.v.weight', 'decoder.block.23.layer.1.layer_norm.weight', 'decoder.block.15.layer.0.SelfAttention.k.weight', 'decoder.block.18.layer.1.layer_norm.weight', 'decoder.block.18.layer.1.EncDecAttention.o.weight', 'decoder.block.3.layer.1.EncDecAttention.k.weight', 'decoder.block.22.layer.2.layer_norm.weight', 'decoder.block.14.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.11.layer.2.DenseReluDense.wo.weight', 'decoder.block.20.layer.1.EncDecAttention.k.weight', 'decoder.block.6.layer.0.SelfAttention.k.weight', 'decoder.block.20.layer.1.layer_norm.weight', 'decoder.block.8.layer.1.Enc

In [39]:
# Initialize protein sequences (taken from two PH domain proteins)
seq1 = 'AGFISVISKKQGEYLEDEWY'
seq2 = 'QVLDKFGS'
#seq1 = 'MRSSASRLSSFSSRDSLWNRMPDQISVSEFIAETTEDYNSPTTSSFTTRLHNCRNTVTLLEEALDQDRTALQKVKKSVKAIYNSGQDHVQNEENYAQVLDKFGSNFLSRDNPDLGTAFVKFSTLTKELSTLLKNLLQGLSHNVIFTLDSLLKGDLKGVKGDLKKPFDKAWKDYETKFTKIEKEKREHAKQHGMIRTEITGAEIAEEMEKERRLFQLQMCEYLIKVNEIKTKKGVDLLQNLIKYYHAQCNFFQDGLKTADKLKQYIEKLAADLYNIKQTQDEEKKQLTALRDLIKSSLQLDQKEDSQSRQGGYSMHQLQGN'
#seq2 = 'NTVTLLEEALDQDRTALQKVKKSVKAIYNSGQDHVQNEENYAQVLDKFGSNFLSRDNPDLGTAF'

In [40]:
def write_align(seq1, seq2):
    """=============================================================================================
    This function accepts two sequences after gaps have been introduced and writes them to a file
    in no particular format (yet).

    :param seq1: first aligned sequence
    :param seq2: second aligned sequence
    ============================================================================================="""

    with open('alignment.txt', 'w', encoding='utf8') as file:
        file.write('Smith-Waterman Pairwise Sequence Alignment\n\n\n')
        file.write(f'Sequence 1   {seq1}\n')
        file.write(f'Sequence 2    {seq2}')

In [41]:
def vec_seqs(seq1, seq2):
    """=============================================================================================
    This function accepts two protein sequences and returns a two lists of vectors, one list for
    each protein where each vector represents one amino acid. Vectorization performed by the Rostlab
    ProtT5-XL_UniRef50 model.

    :param seq1: first protein sequence
    :param seq2: second protein sequence
    return: lists of vectorized amino acids
    ============================================================================================="""

    # Tokenize, encode, and load sequences
    sequences = [' '.join([*seq1]), ' '.join([*seq2])]  # Add spaces between each amino acid
    sequences = [re.sub(r"[UZOB]", "X", seq) for seq in sequences]
    ids = tokenizer.batch_encode_plus(sequences, add_special_tokens=True, padding=True)
    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)

    # Extract sequence features
    with torch.no_grad():
        embedding = model(input_ids=input_ids,attention_mask=attention_mask)
    embedding = embedding.last_hidden_state.cpu().numpy()

    # Remove padding and special tokens
    features = [] 
    for seq_num in range(len(embedding)):
        seq_len = (attention_mask[seq_num] == 1).sum()
        seq_emd = embedding[seq_num][:seq_len-1]
        features.append(seq_emd)

    # Return lists of vectors
    seq1 = features[0]
    seq2 = features[1]
    return seq1, seq2

In [42]:
def SW_align(seq1, seq2, vecs1, vecs2):
    """=============================================================================================
    This function accepts two sequences, creates a matrix corresponding to their lengths, and  
    calculates the score of the alignments for each index. A second matrix is scored so that the
    best alignment can be tracebacked.

    :param seq1: first sequence
    :param seq2: second sequence
    :param vecs1: list of vectorized amino acids for first sequence
    :param vecs2: list of vectorized amino acids for second sequence
    return: scoring and traceback matrices of optimal scores for the SW-alignment of sequences
    ============================================================================================="""

    # NCBI default gap costs
    gap_open = -11
    gap_ext = -1
    gap = False

    # Protein alphabet
    chars = 'ACDEFGHIKLMNPQRSTVWY'

    # Initialize scoring and traceback matrix based on sequence lengths
    row_length = len(seq1)+1
    col_length = len(seq2)+1
    score_m = np.full((row_length, col_length), 0)
    trace_m = np.full((row_length, col_length), 0)

    # Score matrix by moving through each index
    for i in range(len(seq1)):
        seq1_char = seq1[i]  # Character in 1st sequence
        seq1_vec = vecs1[i]  # Corresponding amino acid vector
        for j in range(len(seq2)):
            seq2_char = seq2[j]
            
            # Preceding scoring matrix values
            diagonal = score_m[i][j]
            horizontal = score_m[i+1][j]
            vertical = score_m[i][j+1]

            # Score residues based off cosine similarity between vectors
            seq2_vec = vecs2[j]  # Corresponding amino acid vector
            cos_sim = np.dot(seq1_vec,seq2_vec)/(np.linalg.norm(seq1_vec)*np.linalg.norm(seq2_vec))

            '''TRYING OUT DIFFERENT COSINE SIMILARITY VALUE MODIFIERS'''
            cos_sim = (cos_sim*10)

            # Add to matrix values via scoring method
            diagonal += cos_sim
            if gap is False:  # Apply gap_open penalty if there is no gap
                horizontal += gap_open
                vertical += gap_open
            if gap is True:  # Apply gap_extension penalty if there is a gap
                horizontal += gap_ext
                vertical += gap_ext

            # Update gap status
            score = max(diagonal, horizontal, vertical)
            if score == horizontal: gap = True
            if score == vertical: gap = True
            if score == diagonal: gap = False

            # Assign value to traceback matrix
            if score == diagonal:
                trace_m[i+1][j+1] = 0
            if score == horizontal:
                trace_m[i+1][j+1] = -1
            if score == vertical:
                trace_m[i+1][j+1] = -1

            # Assign max value to scoring matrix
            score_m[i+1][j+1] = max(score, 0)

    return score_m, trace_m

In [43]:
# Call vec_seqs() to get list of vectorized amino acids
seq1_vecs, seq2_vecs = vec_seqs(seq1, seq2)

In [44]:
# Call SW_align()
score_m, trace_m = SW_align(seq1, seq2, seq1_vecs, seq2_vecs)

In [45]:
score_m

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  5,  3,  2,  1,  1,  1,  0,  0],
       [ 0,  3,  8,  4,  3,  2,  1,  2,  0],
       [ 0,  2,  6, 11,  5,  4,  5,  1,  2],
       [ 0,  2,  4, 10, 12,  6,  5,  6,  1],
       [ 0,  3,  3,  6, 12, 13,  7,  6,  7],
       [ 0,  2,  6,  5,  7, 13, 15,  7,  6],
       [ 0,  1,  4,  9,  6,  8, 15, 16,  7],
       [ 0,  1,  1,  5, 11,  7,  9, 16, 17],
       [ 0,  2,  1,  1,  7, 15,  8, 10, 17],
       [ 0,  1,  2,  1,  2, 11, 16,  9, 11],
       [ 0,  2,  2,  2,  1,  3, 12, 17, 10],
       [ 0,  0,  3,  4,  3,  1,  4, 14, 17],
       [ 0,  1,  0,  4,  7,  6,  1,  5, 15],
       [ 0,  1,  3,  1,  5,  8,  8,  1,  5],
       [ 0,  1,  3,  7,  2,  6, 11,  9,  1],
       [ 0,  1,  2,  5,  9,  4,  7, 13, 10],
       [ 0,  0,  1,  3,  9, 11,  6,  9, 14],
       [ 0,  0,  0,  2,  5, 12, 13,  9, 11],
       [ 0,  0,  1,  1,  3,  7, 15, 17, 11],
       [ 0,  0,  0,  1,  1,  4,  9, 18, 23]])

In [46]:
def traceback(score_m, trace_m, seq1, seq2):
    """=============================================================================================
    This function accepts a scoring and a traceback matrix and two sequences and returns the highest 
    scoring local alignment between the two sequences

    :param score_m: scoring matrix
    :param trace_m: traceback matrix
    :param seq1: first sequence
    :param seq2: second sequence
    return: highest scoring local alignment of the two sequences
    ============================================================================================="""

    # Find index of highest score in scoring matrix, start traceback at this matrix
    high_score_ind = np.unravel_index(np.argmax(score_m, axis=None), score_m.shape)

    # Reverse strings and convert to lists so gaps can be inserted
    rev_seq1 = list(seq1[::-1])
    rev_seq2 = list(seq2[::-1])

    # Move through matrix starting at bottom right
    index = [high_score_ind[0], high_score_ind[1]]
    count = 0
    while (index[0] and index[1]) != 0:
        val = trace_m[index[0], index[1]]

        if val == 1:  # If cell is equal to 1, insert a gap into the second sequence
            index[0] = index[0] - 1
            rev_seq2.insert(count, '_')
        if val == -1:  # If cell is equal to -1, insert a gap into the first sequence
            index[1] = index[1] - 1
            rev_seq1.insert(count, '_')
        if val == 0:  # If cell is equal to 0, there is no gap
            index[0] = index[0] - 1
            index[1] = index[1] - 1
        count += 1

    # Join lists and reverse strings again
    seq1 = ''.join(rev_seq1)
    seq2 = ''.join(rev_seq2)

    # Need to store alignment data somehow, but until then we are printing spaces
    space = ' '
    print(index[1]*space+seq1[::-1])
    print(index[0]*space+seq2[::-1]) 

In [47]:
traceback(score_m, trace_m, seq1, seq2)

AGFISVISKKQGEYLEDEWY
            QVLDKFGS
